In [22]:
from keras.preprocessing.image import ImageDataGenerator

In [23]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
x_train = train_datagen.flow_from_directory('../input/gesture-images/train', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('../input/gesture-images/test', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')


Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [25]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [26]:
classifier=Sequential()

In [27]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1) ,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

In [28]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))

In [29]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_3 (Dense)              (None, 6)                

In [30]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
classifier.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=20, validation_data=x_test, validation_steps=len(x_test)
)

Epoch 1/20
119/119 [==============================] - 3s 18ms/step - loss: 1.4617 - accuracy: 0.3872 - val_loss: 0.8718 - val_accuracy: 0.5333
Epoch 2/20
119/119 [==============================] - 2s 18ms/step - loss: 0.7466 - accuracy: 0.6970 - val_loss: 0.8525 - val_accuracy: 0.7000
Epoch 3/20
119/119 [==============================] - 2s 17ms/step - loss: 0.4967 - accuracy: 0.8165 - val_loss: 0.3680 - val_accuracy: 0.9000
Epoch 4/20
119/119 [==============================] - 2s 18ms/step - loss: 0.3246 - accuracy: 0.8771 - val_loss: 0.4205 - val_accuracy: 0.9333
Epoch 5/20
119/119 [==============================] - 2s 17ms/step - loss: 0.2229 - accuracy: 0.9226 - val_loss: 0.3529 - val_accuracy: 0.9000
Epoch 6/20
119/119 [==============================] - 2s 16ms/step - loss: 0.2133 - accuracy: 0.9259 - val_loss: 0.2520 - val_accuracy: 0.8667
Epoch 7/20
119/119 [==============================] - 2s 16ms/step - loss: 0.1496 - accuracy: 0.9444 - val_loss: 0.3539 - val_accuracy: 0.8667

In [32]:
classifier.save('gesture.h5')

In [33]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as f:
    f.write(model_json)

In [34]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("gesture.h5")

In [35]:
from tensorflow.keras.utils import load_img, img_to_array

img = load_img("../input/gesture-images/test/1/0.jpg", grayscale=True, target_size=(64, 64))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)

pred

array([[0., 1., 0., 0., 0., 0.]], dtype=float32)

In [40]:
pred[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)